# Set up

In [1]:
import glob # for system ls
from natsort import natsorted # for alphabetical sorting

import xarray as xr # for reading netcdf
# import numpy as np
import dask.array as da
import dask
import rioxarray as rio # for writing tif

import os

In [6]:
experiment='Hist'
year=2020

# input data directories are the same for everyone on HPC Orion
# data_nc = '/work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/'
# data_static = '/work/hpc/datasets/un_fao/pyaez/static/netcdf/'
data_nc = '/work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/'
data_static = '/work/hpc/datasets/un_fao/gaez_v5_intermediate/static/'


# these are output directories, change per user or overwrite files at these locations
# if you change these, make sure the directories exist (mkdir) before running the script
# data_npy = '/work/hpc/datasets/un_fao/pyaez/global_daily/'
# data_tif = '/work/hpc/datasets/un_fao/pyaez/global_static/'
data_npy = '/work/hpc/datasets/un_fao/pyaez/'
data_tif = '/work/hpc/datasets/un_fao/pyaez/'

countryinfo={'Mozambique':{'lat1':-9,'lat2':-29,'lon1':29,'lon2':43},
             'Pakistan':{'lat1':39,'lat2':22,'lon1':60,'lon2':77},
             'Tunisia':{'lat1':39,'lat2':29,'lon1':6,'lon2':13}}

varnames = ['Precip','Rhum','Srad','Tmax-2m','Tmin-2m','Wind-2m']
varnames365 = ['Precip365','Rhum365','Srad365','Tmax-2m365','Tmin-2m365','Wind-2m365']

# Main Code

# netcdf to npy

In [54]:
# each 3D array has total size ~12.7GB
# we need to chunk these arrays so that a chunk fits into memory (~9GB per single core)
# any chunk size less than ~9GB should work, we'll use 8 chunks --> ~1.5GB per chunk

# chunks={'time':-1,'lat':450,'lon':2160} # 8 chunks
chunks={'time':-1,'lat':200,'lon':-1}

# look at size of the chunks
test=xr.open_dataset(data_nc+'Tmin-2m_'+experiment+'_daily_'+str(year)+'_5m.nc')['Tmin-2m']
# test

In [55]:
c='Mozambique'
test=test.sel(lat=slice(countryinfo[c]['lat1'],countryinfo[c]['lat2']),
              lon=slice(countryinfo[c]['lon1'],countryinfo[c]['lon2']))
dropdate=str(year)+'-02-29'
test=test.drop_sel(time=dropdate)
test

<xarray.DataArray 'Tmin-2m' (time: 365, lat: 240, lon: 168)>
[14716800 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
  * lat          (lat) float32 -9.042 -9.125 -9.208 ... -28.79 -28.88 -28.96
  * lon          (lon) float32 29.04 29.12 29.21 29.29 ... 42.79 42.88 42.96
    spatial_ref  (time) int64 ...
Attributes:
    standard_name:  Tmin-2m
    long_name:      2m minimum surface air temperature
    units:          degrees C

In [42]:
c='Tunisia'
data = xr.open_dataset(f,chunks=chunks)[var_in].sel(lat=slice(countryinfo[c]['lat1'],countryinfo[c]['lat2']),\
                        lon=slice(countryinfo[c]['lon1'],countryinfo[c]['lon2'])).drop_sel(time=dropdate).transpose('lat','lon','time').data
# del data
# data

KeyError: 'Wind-2m'

In [ ]:
# # this cell outputs all 366 days of data, skip for now

# del test

# # do the nc to npy conversion for each variable
# # we are also subsetting the global array (to eliminate artarctica where all grids are nan)
# # expect about 1 min run time per variable

# for var in varnames:
#     # get nc file name
#     f = natsorted(glob.glob(data_nc+var+'*_5m.nc'))[0]
        
#     if f:
#         # read netcdf data into a dask array of numpy array chunks
#         print('reading',f)
#         data = xr.open_dataset(f,chunks=chunks)[var].sel(lat=slice(90,-60.)).transpose('lat','lon','time').data        

#         # set up dir for writing npy
#         out_dir=data_npy+var+'/'
#         isExist = os.path.exists(out_dir)
#         if not isExist:
#             os.makedirs(out_dir)
#         # write npy data
#         print('writing to',out_dir+'0.npy')     
#         da.to_npy_stack(out_dir,data,axis=2)          
#     else:
#         print('no file',f)

In [25]:
# to output files without the leap day run this

# do the nc to npy conversion for each variable
# we are also subsetting the global array (to eliminate artarctica where all grids are nan)
# we also drop the leap day to output 365 total days
# expect about 1 min run time per variable

# del test

c='Mozambique'

with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    for var_in,var_out in zip(varnames,varnames365):
        # get nc file name
#         f = natsorted(glob.glob(data_nc+var_in+'*_5m.nc'))[0]
        f = glob.glob(data_nc+var_in+'_'+experiment+'_daily_'+str(year)+'_5m.nc')[0]

        if f:
            # read netcdf data into a dask array of numpy array chunks
            print('reading',f)
            try:
                dropdate=str(year)+'-02-29'
                data = xr.open_dataset(f,chunks=chunks)[var_in].sel(lat=slice(countryinfo[c]['lat1'],countryinfo[c]['lat2']),\
                        lon=slice(countryinfo[c]['lon1'],countryinfo[c]['lon2'])).drop_sel(time=dropdate).transpose('lat','lon','time').data
            except:
                data = xr.open_dataset(f,chunks=chunks)[var_in].sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2)).transpose('lat','lon','time').data

            # set up dir for writing npy
            out_dir=data_npy+c+'_daily/'+str(year)+'/'+var_out+'/'
            isExist = os.path.exists(out_dir)
            if not isExist:
                os.makedirs(out_dir)
            # write npy data
            print('writing to',out_dir+'0.npy')     
            da.to_npy_stack(out_dir,data,axis=2)          
        else:
            print('no file',f)

reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/Precip_Hist_daily_2020_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/Mozambique_daily/2020/Precip365/0.npy
reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/Rhum_Hist_daily_2020_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/Mozambique_daily/2020/Rhum365/0.npy
reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/Srad_Hist_daily_2020_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/Mozambique_daily/2020/Srad365/0.npy
reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/Tmax-2m_Hist_daily_2020_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/Mozambique_daily/2020/Tmax-2m365/0.npy
reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/Tmin-2m_Hist_daily_2020_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/Mozambique_daily/2020/Tmin-2m365/0.npy
reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/netcdf/Wind-2m_Hist_daily_2020_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/Mozambiqu

# netcdf mask and elevation to tif

### mask

In [56]:
# get file path and file name
# f = glob.glob(data_static+'mask_*_5m.nc')[0]
f = glob.glob(data_static+'mask_2268708_5m.nc')[0]
filename=f.split('/')[-1]
print(f)
print(filename)

/work/hpc/datasets/un_fao/gaez_v5_intermediate/static/mask_2268708_5m.nc
mask_2268708_5m.nc


In [58]:
print('reading',f)
c='Mozambique'
# load mask from netcdf file, remove antarctica
data = rio.open_rasterio(f).sel(y=slice(countryinfo[c]['lat1'],countryinfo[c]['lat2']),\
                        x=slice(countryinfo[c]['lon1'],countryinfo[c]['lon2']))

# write file
outdir=data_tif+c+'_static/'
outfile=filename[:-3]+'.tif'
isExist = os.path.exists(outdir)
if not isExist:
    os.makedirs(outdir)
print('writing',outdir+outfile)
data.rio.to_raster(outdir+outfile)

data


reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/static/mask_2268708_5m.nc
writing /work/hpc/datasets/un_fao/pyaez/Mozambique_static/mask_2268708_5m.tif


<xarray.DataArray 'mask' (band: 1, y: 240, x: 168)>
[40320 values with dtype=int32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 29.04 29.12 29.21 29.29 ... 42.71 42.79 42.87 42.96
  * y            (y) float64 -9.042 -9.125 -9.208 ... -28.79 -28.87 -28.96
    spatial_ref  int64 0
Attributes: (12/16)
    lat#axis:           Y
    lat#long_name:      latitude
    lat#standard_name:  latitude
    lat#units:          degrees_north
    lon#axis:           X
    lon#long_name:      longitude
    ...                 ...
    long_name:          mask
    standard_name:      mask
    units:              unitless
    _FillValue:         -2147483647
    scale_factor:       1.0
    add_offset:         0.0

### elevation

In [59]:
# get file path and file name
f = glob.glob(data_static+'elev_2268708_5m.nc')[0]
filename=f.split('/')[-1]
print(f)
print(filename)

/work/hpc/datasets/un_fao/gaez_v5_intermediate/static/elev_2268708_5m.nc
elev_2268708_5m.nc


In [60]:
print('reading',f)

# load mask from netcdf file
data = rio.open_rasterio(f).sel(y=slice(countryinfo[c]['lat1'],countryinfo[c]['lat2']),\
                        x=slice(countryinfo[c]['lon1'],countryinfo[c]['lon2']))
# write file
outdir=data_tif+c+'_static/'
outfile=filename[:-3]+'.tif'
isExist = os.path.exists(outdir)
if not isExist:
    os.makedirs(outdir)
print('writing',outdir+outfile)
data.rio.to_raster(outdir+outfile)

data


reading /work/hpc/datasets/un_fao/gaez_v5_intermediate/static/elev_2268708_5m.nc
writing /work/hpc/datasets/un_fao/pyaez/Mozambique_static/elev_2268708_5m.tif


<xarray.DataArray 'elev' (band: 1, y: 240, x: 168)>
[40320 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 29.04 29.12 29.21 29.29 ... 42.71 42.79 42.87 42.96
  * y            (y) float64 -9.042 -9.125 -9.208 ... -28.79 -28.87 -28.96
    spatial_ref  int64 0
Attributes: (12/16)
    lat#axis:           Y
    lat#long_name:      latitude
    lat#standard_name:  latitude
    lat#units:          degrees_north
    lon#axis:           X
    lon#long_name:      longitude
    ...                 ...
    long_name:          elevation
    standard_name:      elev
    units:              m
    _FillValue:         nan
    scale_factor:       1.0
    add_offset:         0.0

## soil

In [61]:
# get file path and file name
f = glob.glob('/work/hpc/datasets/un_fao/pyaez/global_static/soil_terrain_lulc_global_08333.tif')[0]
filename=f.split('/')[-1]
print(f)
print(filename)

/work/hpc/datasets/un_fao/pyaez/global_static/soil_terrain_lulc_global_08333.tif
soil_terrain_lulc_global_08333.tif


In [62]:
print('reading',f)

# load mask from netcdf file
data = rio.open_rasterio(f).sel(y=slice(countryinfo[c]['lat1'],countryinfo[c]['lat2']),\
                        x=slice(countryinfo[c]['lon1'],countryinfo[c]['lon2']))
# write file
outdir=data_tif+c+'_static/'
outfile=filename
isExist = os.path.exists(outdir)
if not isExist:
    os.makedirs(outdir)
print('writing',outdir+outfile)
data.rio.to_raster(outdir+outfile)

data


reading /work/hpc/datasets/un_fao/pyaez/global_static/soil_terrain_lulc_global_08333.tif
writing /work/hpc/datasets/un_fao/pyaez/Mozambique_static/soil_terrain_lulc_global_08333.tif


<xarray.DataArray (band: 1, y: 240, x: 168)>
[40320 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 29.04 29.12 29.21 29.29 ... 42.71 42.79 42.87 42.96
  * y            (y) float64 -9.042 -9.125 -9.208 ... -28.79 -28.88 -28.96
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    DataType:            Thematic
    RepresentationType:  THEMATIC
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0